<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Parameter server end-to-end sample (Training part)

## Overview

In HugeCTR version 3.3, we finished the whole pipeline of parameter server, including 
1. The parameter dumping interface from training to kafka.
2. Redis as a level2 cache.
3. RocksDB as a persistence storage.
4. Embedding cache update mechanism.


The purpose of this notebook is to give you a brief idea of how parameter server works in terms of the flow of parameters. 

## Table of Contents
-  [Data Preparation](#1)
-  [Data Preprocessing using NVTabular](#2)
-  [Kafka broker start](#3)
-  [Wide&Deep Continuous Training Demo](#4)

## Data preparation

1. Firstly, we made a folder to store our data:

In [ ]:
!mkdir criteo_data

2. Download Criteo Dataset

In [ ]:
!wget http://azuremlsampleexperiments.blob.core.windows.net/criteo/day_1.gz

**NOTE**: Replace `1` with a value from [0, 23] to use a different day.

During preprocessing, the amount of data, which is used to speed up the preprocessing, fill missing values, and remove the feature values that are considered rare, is further reduced.

3. Thirdly, preprocess the data and split the data into 6 parts. 

In [1]:
%%writefile preprocess.sh

#!/bin/bash

DST_DATA_DIR=$2

echo "Getting the first few examples from the uncompressed dataset..."

head -n 420000 day_$1 > $DST_DATA_DIR/day_$1_small
if [ $? -ne 0 ]; then
	echo "Warning: fallback to find original compressed data day_$1.gz..."
	echo "Decompressing day_$1.gz..."
	gzip -d -c day_$1.gz > day_$1
	if [ $? -ne 0 ]; then
		echo "Error: failed to decompress the file."
		exit 2
	fi
	head -n 420000 day_$1 > $DST_DATA_DIR/day_$1_small
	if [ $? -ne 0 ]; then
		echo "Error: day_$1 file"
		exit 2
	fi
fi

echo "Counting the number of samples in day_$1 dataset..."
total_count=$(wc -l $DST_DATA_DIR/day_$1_small)
total_count=(${total_count})
echo "The first $total_count examples will be used in day_$1 dataset."

echo "Shuffling dataset..."
shuf $DST_DATA_DIR/day_$1_small > $DST_DATA_DIR/day_$1_shuf

echo "Dividing data into 6 parts..."
cd $DST_DATA_DIR
split -n l/6 day_$1_shuf

echo "Done!"

Overwriting preprocess.sh


4. Run the preprocess script

In [2]:
!bash preprocess.sh 1 criteo_data

Getting the first few examples from the uncompressed dataset...
Counting the number of samples in day_1 dataset...
The first 420000 examples will be used in day_1 dataset.
Shuffling dataset...
Dividing data into 6 parts...
Done!


**IMPORTANT NOTES**: 
- The first argument represents the dataset postfix.  For instance, if `day_1` is used, the postfix is `1`.
- The second argument, `criteo_data`, is where the preprocessed data is stored.

## Preprocess using NVTabular

In [3]:
%%writefile preprocess.py

import os
import sys
import argparse
import glob
import time
import numpy as np
import pandas as pd
import concurrent.futures as cf
from concurrent.futures import as_completed
import shutil

import dask_cudf
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask.utils import parse_bytes
from dask.delayed import delayed

import cudf
import rmm
import nvtabular as nvt
from nvtabular.io import Shuffle
from nvtabular.utils import device_mem_size
from nvtabular.ops import Categorify, Clip, FillMissing, HashBucket, LambdaOp, Normalize, Rename, Operator, get_embedding_sizes
#%load_ext memory_profiler

import logging
logging.basicConfig(format='%(asctime)s %(message)s')
logging.root.setLevel(logging.NOTSET)
logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('asyncio').setLevel(logging.WARNING)

# define dataset schema
CATEGORICAL_COLUMNS=["C" + str(x) for x in range(1, 27)]
CONTINUOUS_COLUMNS=["I" + str(x) for x in range(1, 14)]
LABEL_COLUMNS = ['label']
COLUMNS =  LABEL_COLUMNS + CONTINUOUS_COLUMNS +  CATEGORICAL_COLUMNS
#/samples/criteo mode doesn't have dense features
criteo_COLUMN=LABEL_COLUMNS +  CATEGORICAL_COLUMNS
#For new feature cross columns
CROSS_COLUMNS = []


NUM_INTEGER_COLUMNS = 13
NUM_CATEGORICAL_COLUMNS = 26
NUM_TOTAL_COLUMNS = 1 + NUM_INTEGER_COLUMNS + NUM_CATEGORICAL_COLUMNS


# Initialize RMM pool on ALL workers
def setup_rmm_pool(client, pool_size):
    client.run(rmm.reinitialize, pool_allocator=True, initial_pool_size=pool_size)
    return None

#compute the partition size with GB
def bytesto(bytes, to, bsize=1024):
    a = {'k' : 1, 'm': 2, 'g' : 3, 't' : 4, 'p' : 5, 'e' : 6 }
    r = float(bytes)
    return bytes / (bsize ** a[to])

class FeatureCross(Operator):
    def __init__(self, dependency):
        self.dependency = dependency

    def transform(self, columns, gdf):
        new_df = type(gdf)()
        for col in columns:
            new_df[col] = gdf[col] + gdf[self.dependency]
        return new_df

    def dependencies(self):
        return [self.dependency]

#process the data with NVTabular
def process_NVT(args):

    if args.feature_cross_list:
        feature_pairs = [pair.split("_") for pair in args.feature_cross_list.split(",")]
        for pair in feature_pairs:
            CROSS_COLUMNS.append(pair[0]+'_'+pair[1])


    logging.info('NVTabular processing')
    
    ### TODO
    files = ["day_1_shuf", "xaa", "xab", "xac", "xad", "xae", "xaf"]
    inputs = [os.path.join(args.data_path, file) for file in files]

    ## Get Dask Client

    # Deploy a Single-Machine Multi-GPU Cluster
    device_size = device_mem_size(kind="total")
    cluster = None
    if args.protocol == "ucx":
        UCX_TLS = os.environ.get("UCX_TLS", "tcp,cuda_copy,cuda_ipc,sockcm")
        os.environ["UCX_TLS"] = UCX_TLS
        cluster = LocalCUDACluster(
            protocol = args.protocol,
            CUDA_VISIBLE_DEVICES = args.devices,
            n_workers = len(args.devices.split(",")),
            enable_nvlink=True,
            device_memory_limit = int(device_size * args.device_limit_frac),
            dashboard_address=":" + args.dashboard_port
        )
    else:
        cluster = LocalCUDACluster(
            protocol = args.protocol,
            n_workers = len(args.devices.split(",")),
            CUDA_VISIBLE_DEVICES = args.devices,
            device_memory_limit = int(device_size * args.device_limit_frac),
            dashboard_address=":" + args.dashboard_port
        )



    # Create the distributed client
    client = Client(cluster)
    if args.device_pool_frac > 0.01:
        setup_rmm_pool(client, int(args.device_pool_frac*device_size))


    #calculate the total processing time
    runtime = time.time()

    #test dataset without the label feature
    if args.dataset_type == 'test':
        global LABEL_COLUMNS
        LABEL_COLUMNS = []

    ##-----------------------------------##
    # Dask rapids converts txt to parquet
    # Dask cudf dataframe = ddf

    for i in inputs:

        ddf = dask_cudf.read_csv(i, sep='\t',names=LABEL_COLUMNS + CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS)

        ## Convert label col to FP32
        if args.parquet_format and args.dataset_type == 'train':
            ddf["label"] = ddf['label'].astype('float32')

        # Save it as parquet format for better memory usage
        ddf.to_parquet(i + "_temp", header=True)
        ##-----------------------------------##

    COLUMNS =  LABEL_COLUMNS + CONTINUOUS_COLUMNS + CROSS_COLUMNS + CATEGORICAL_COLUMNS
    
    
    paths = glob.glob(os.path.join(args.data_path, "*_temp"))

    categorify_op = Categorify(freq_threshold=args.freq_limit)
    cat_features = CATEGORICAL_COLUMNS >> categorify_op
    cont_features = CONTINUOUS_COLUMNS >> FillMissing() >> Clip(min_value=0) >> Normalize()
    cross_cat_op = Categorify(freq_threshold=args.freq_limit)

    features = LABEL_COLUMNS
    
    if args.criteo_mode == 0:
        features += cont_features
        if args.feature_cross_list:
            feature_pairs = [pair.split("_") for pair in args.feature_cross_list.split(",")]
            for pair in feature_pairs:
                col0 = pair[0]
                col1 = pair[1]
                features += col0 >> FeatureCross(col1)  >> Rename(postfix="_"+col1) >> cross_cat_op
            
    features += cat_features

    workflow = nvt.Workflow(features, client=client)

    logging.info("Preprocessing")

    output_format = 'hugectr'
    if args.parquet_format:
        output_format = 'parquet'

    # just for /samples/criteo model
    iterators = []
    for path in paths:
        iterators.append(nvt.Dataset(path, engine='parquet', part_size=int(args.part_mem_frac * device_size)))

    shuffle = None
    if args.shuffle == "PER_WORKER":
        shuffle = nvt.io.Shuffle.PER_WORKER
    elif args.shuffle == "PER_PARTITION":
        shuffle = nvt.io.Shuffle.PER_PARTITION

    logging.info('Preprocessing.....')

    dict_dtypes = {}
    for col in CATEGORICAL_COLUMNS:
        dict_dtypes[col] = np.int64
    if not args.criteo_mode:
        for col in CONTINUOUS_COLUMNS:
            dict_dtypes[col] = np.float32
    for col in CROSS_COLUMNS:
        dict_dtypes[col] = np.int64
    for col in LABEL_COLUMNS:
        dict_dtypes[col] = np.float32
    
    conts = CONTINUOUS_COLUMNS if not args.criteo_mode else []
    
    workflow.fit(iterators[0])
     
    i=0
    for iterator in iterators[1:]:
        if output_format == 'hugectr':
            workflow.transform(iterator).to_hugectr(
                    cats=CATEGORICAL_COLUMNS + CROSS_COLUMNS,
                    conts=conts,
                    labels=LABEL_COLUMNS,
                    output_path=args.out_path + '/file' + str(i),
                    shuffle=shuffle,
                    out_files_per_proc=args.out_files_per_proc,
                    num_threads=args.num_io_threads,
                    write_hugectr_keyset=True)
        else:
            workflow.transform(iterator).to_parquet(
                    output_path=args.out_path + '/file' + str(i),
                    dtypes=dict_dtypes,
                    cats=CATEGORICAL_COLUMNS + CROSS_COLUMNS,
                    conts=conts,
                    labels=LABEL_COLUMNS,
                    shuffle=shuffle,
                    out_files_per_proc=args.out_files_per_proc,
                    num_threads=args.num_io_threads,
                    write_hugectr_keyset=True)
        i += 1

        embeddings_dict_cat = categorify_op.get_embedding_sizes(CATEGORICAL_COLUMNS)
        embeddings_dict_cross = cross_cat_op.get_embedding_sizes(CROSS_COLUMNS)
        embeddings = [embeddings_dict_cross[c][0] for c in CROSS_COLUMNS] + [embeddings_dict_cat[c][0] for c in CATEGORICAL_COLUMNS]

        print(embeddings)
    ##--------------------##

    ## Shutdown clusters
    client.close()
    logging.info('NVTabular processing done')

    runtime = time.time() - runtime

    print("\nDask-NVTabular Criteo Preprocessing")
    print("--------------------------------------")
    print(f"data_path          | {args.data_path}")
    print(f"output_path        | {args.out_path}")
    print(f"partition size     | {'%.2f GB'%bytesto(int(args.part_mem_frac * device_size),'g')}")
    print(f"protocol           | {args.protocol}")
    print(f"device(s)          | {args.devices}")
    print(f"rmm-pool-frac      | {(args.device_pool_frac)}")
    print(f"out-files-per-proc | {args.out_files_per_proc}")
    print(f"num_io_threads     | {args.num_io_threads}")
    print(f"shuffle            | {args.shuffle}")
    print("======================================")
    print(f"Runtime[s]         | {runtime}")
    print("======================================\n")


def parse_args():
    parser = argparse.ArgumentParser(description=("Multi-GPU Criteo Preprocessing"))

    #
    # System Options
    #

    parser.add_argument("--data_path", type=str, help="Input dataset path (Required)")
    parser.add_argument("--out_path", type=str, help="Directory path to write output (Required)")
    parser.add_argument(
        "-d",
        "--devices",
        default=os.environ.get("CUDA_VISIBLE_DEVICES", "0"),
        type=str,
        help='Comma-separated list of visible devices (e.g. "0,1,2,3"). '
    )
    parser.add_argument(
        "-p",
        "--protocol",
        choices=["tcp", "ucx"],
        default="tcp",
        type=str,
        help="Communication protocol to use (Default 'tcp')",
    )
    parser.add_argument(
        "--device_limit_frac",
        default=0.5,
        type=float,
        help="Worker device-memory limit as a fraction of GPU capacity (Default 0.8). "
    )
    parser.add_argument(
        "--device_pool_frac",
        default=0.9,
        type=float,
        help="RMM pool size for each worker  as a fraction of GPU capacity (Default 0.9). "
        "The RMM pool frac is the same for all GPUs, make sure each one has enough memory size",
    )
    parser.add_argument(
        "--num_io_threads",
        default=0,
        type=int,
        help="Number of threads to use when writing output data (Default 0). "
        "If 0 is specified, multi-threading will not be used for IO.",
    )

    #
    # Data-Decomposition Parameters
    #

    parser.add_argument(
        "--part_mem_frac",
        default=0.125,
        type=float,
        help="Maximum size desired for dataset partitions as a fraction "
        "of GPU capacity (Default 0.125)",
    )
    parser.add_argument(
        "--out_files_per_proc",
        default=8,
        type=int,
        help="Number of output files to write on each worker (Default 8)",
    )

    #
    # Preprocessing Options
    #

    parser.add_argument(
        "-f",
        "--freq_limit",
        default=0,
        type=int,
        help="Frequency limit for categorical encoding (Default 0)",
    )
    parser.add_argument(
        "-s",
        "--shuffle",
        choices=["PER_WORKER", "PER_PARTITION", "NONE"],
        default="PER_PARTITION",
        help="Shuffle algorithm to use when writing output data to disk (Default PER_PARTITION)",
    )

    parser.add_argument(
        "--feature_cross_list", default=None, type=str, help="List of feature crossing cols (e.g. C1_C2, C3_C4)"
    )

    #
    # Diagnostics Options
    #

    parser.add_argument(
        "--profile",
        metavar="PATH",
        default=None,
        type=str,
        help="Specify a file path to export a Dask profile report (E.g. dask-report.html)."
        "If this option is excluded from the command, not profile will be exported",
    )
    parser.add_argument(
        "--dashboard_port",
        default="8787",
        type=str,
        help="Specify the desired port of Dask's diagnostics-dashboard (Default `3787`). "
        "The dashboard will be hosted at http://<IP>:<PORT>/status",
    )

    #
    # Format
    #

    parser.add_argument('--criteo_mode', type=int, default=0)
    parser.add_argument('--parquet_format', type=int, default=1)
    parser.add_argument('--dataset_type', type=str, default='train')

    args = parser.parse_args()
    args.n_workers = len(args.devices.split(","))
    return args
if __name__ == '__main__':

    args = parse_args()

    process_NVT(args)

Overwriting preprocess.py


In [4]:
!python preprocess.py --data_path ./criteo_data --out_path ./criteo_data --freq_limit 6 --device_limit_frac 0.5 --device_pool_frac 0.5 --out_files_per_proc 8 --devices "0"  --num_io_threads 2

2021-11-25 03:28:18,637 NVTabular processing
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2021-11-25 03:28:25,834 Preprocessing
2021-11-25 03:28:26,329 Preprocessing.....
/usr/local/lib/python3.8/dist-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (160) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
[517, 1172, 2057, 393, 688, 3, 1604, 482, 21, 623, 654, 1687, 10, 267, 869, 28, 4, 119, 13, 496, 698, 547, 577, 1683, 27, 24]
[517, 1172, 2057, 393, 688, 3, 1604, 482, 21, 623, 654, 1687, 10, 267, 869, 28, 4, 119, 13, 496, 698, 547, 577, 1683, 27, 24]
[517, 1172, 2057, 393, 688, 3, 1604, 482, 21, 623, 654, 1687, 10, 267, 869, 28, 4, 119, 13, 496, 698, 547, 577, 1683, 27, 24]
[517, 1172, 2057, 393, 688, 3, 1604, 482, 21, 623, 654, 1687, 10, 267, 869, 28, 4, 119, 13, 496, 698, 547, 577, 1683, 27, 24]
[517, 1172, 2057, 393, 688, 3, 1604, 482, 21, 623, 654, 1687, 10

In [ ]:
 !ls -l ./criteo_data/file*

## Kafka broker setup

Install Kafka

```
$ cd /usr/local
$ wget https://dlcdn.apache.org/kafka/3.0.0/kafka_2.12-3.0.0.tgz
$ tar -zxvf kafka_2.12-3.0.0.tgz
$ mv kafka_2.12-3.0.0 kafka

```

Install Zookeeper

```
$ cd /usr/local
$ wget https://dlcdn.apache.org/zookeeper/zookeeper-3.7.0/apache-zookeeper-3.7.0-bin.tar.gz
$ tar -zxvf apache-zookeeper-3.7.0-bin.tar.gz
$ mv apache-zookeeper-3.7.0-bin zookeeper
```

Configure Kafka

**Open /usr/local/kafka/config/server.properties and decomment the following:**
```
listeners = PLAINTEXT://your.host.name:9092
```
**Input your host name and port, this should be consistent with what you input in the training script below.**

Start Zookeeper

```
$ cd /usr/local/zookeeper/bin
$ bash zkServer.sh start

```

Start Kafka

```
$ cd /usr/local/kafka
$ bin/kafka-server-start.sh config/server.properties
```

## Wide&Deep Model Demo

In [11]:
%%writefile wdl_demo.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(model_name = "wdl",
                              max_eval_batches = 5000,
                              batchsize_eval = 1024,
                              batchsize = 1024,
                              lr = 0.001,
                              vvgpu = [[0]],
                              i64_input_key = True,
                              use_mixed_precision = False,
                              repeat_dataset = False,
                              use_cuda_graph = True,
                              kafka_brockers = "10.23.137.25:9093")
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Parquet,
                          source = ["criteo_data/file0/_file_list.txt", "criteo_data/file1/_file_list.txt"],
                          keyset = ["criteo_data/file0/_hugectr.keyset", "criteo_data/file1/_hugectr.keyset"],
                          slot_size_array = [517, 1172, 2057, 393, 688, 3, 1604, 482, 21, 623, 654, 1687, 10, 267, 869, 28, 4, 119, 13, 496, 698, 547, 577, 1683, 27, 24],
                          eval_source = "criteo_data/file2/_file_list.txt",
                          check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
hc_config = hugectr.CreateHMemCache(2, 0.5, 0)
etc = hugectr.CreateETC(ps_types = [hugectr.TrainPSType_t.Staged, hugectr.TrainPSType_t.Cached],\
                        sparse_models = ["./wdl_0_sparse_model", "./wdl_1_sparse_model"],\
                        local_paths = ["raid/md1/temp_dir"], hmem_cache_configs = [hc_config])
model = hugectr.Model(solver, reader, optimizer, etc)
model.add(hugectr.Input(label_dim = 1, label_name = "label",
                        dense_dim = 13, dense_name = "dense",
                        data_reader_sparse_param_array = 
                        [hugectr.DataReaderSparseParam("wide_data", 1, True, 1),
                        hugectr.DataReaderSparseParam("deep_data", 1, True, 25)]))
model.add(hugectr.SparseEmbedding(embedding_type = hugectr.Embedding_t.DistributedSlotSparseEmbeddingHash, 
                            workspace_size_per_gpu_in_mb = 23,
                            embedding_vec_size = 1,
                            combiner = "sum",
                            sparse_embedding_name = "sparse_embedding2",
                            bottom_name = "wide_data",
                            optimizer = optimizer))
model.add(hugectr.SparseEmbedding(embedding_type = hugectr.Embedding_t.DistributedSlotSparseEmbeddingHash, 
                            workspace_size_per_gpu_in_mb = 358,
                            embedding_vec_size = 16,
                            combiner = "sum",
                            sparse_embedding_name = "sparse_embedding1",
                            bottom_name = "deep_data",
                            optimizer = optimizer))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding1"],
                            top_names = ["reshape1"],
                            leading_dim=400))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding2"],
                            top_names = ["reshape2"],
                            leading_dim=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Concat,
                            bottom_names = ["reshape1", "dense"], top_names = ["concat1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["concat1"],
                            top_names = ["fc1"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc1"],
                            top_names = ["relu1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu1"],
                            top_names = ["dropout1"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dropout1"],
                            top_names = ["fc2"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc2"],
                            top_names = ["relu2"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu2"],
                            top_names = ["dropout2"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dropout2"],
                            top_names = ["fc3"],
                            num_output=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Add,
                            bottom_names = ["fc3", "reshape2"],
                            top_names = ["add1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.BinaryCrossEntropyLoss,
                            bottom_names = ["add1", "label"],
                            top_names = ["loss"]))
model.compile()
model.summary()
model.graph_to_json(graph_config_file = "wdl.json")
model.fit(max_iter = 1100, display = 200, eval_interval = 500, snapshot = 1000000, snapshot_prefix = "wdl")
# Get the updated embedding features in model.fit()
# updated_model = model.get_incremental_model()
# User defined operations to the updated_model
# ...+
model.dump_parameters_2kafka()
model.set_source(source = ["criteo_data/file3/_file_list.txt", "criteo_data/file4/_file_list.txt"], keyset = ["criteo_data/file3/_hugectr.keyset", "criteo_data/file4/_hugectr.keyset"], eval_source = "criteo_data/file5/_file_list.txt")
model.fit(max_iter = 1100, display = 200, eval_interval = 500, snapshot = 1000000, snapshot_prefix = "wdl")
# Get the updated embedding features in model.fit()
# updated_model = model.get_incremental_model()
# User defined operations to the updated_model
# ...
model.save_params_to_files("wdl")

Overwriting wdl_demo.py


In [12]:
!rm -r wdl_0_sparse_model
!rm -r wdl_1_sparse_model

In [13]:
!python wdl_demo.py

[HUGECTR][03:09:39][INFO][RANK0]: Empty embedding, trained table will be stored in ./wdl_0_sparse_model
[HUGECTR][03:09:39][INFO][RANK0]: Empty embedding, trained table will be stored in ./wdl_1_sparse_model
HugeCTR Version: 3.2
====================================================Model Init=====================================================
[HUGECTR][03:09:39][INFO][RANK0]: Initialize model: wdl
[HUGECTR][03:09:39][INFO][RANK0]: Global seed is 407180828
[HUGECTR][03:09:40][INFO][RANK0]: Device to NUMA mapping:
  GPU 0 ->  node 0

[HUGECTR][03:09:41][WARNING][RANK0]: Peer-to-peer access cannot be fully enabled.
[HUGECTR][03:09:41][INFO][RANK0]: Start all2all warmup
[HUGECTR][03:09:41][INFO][RANK0]: End all2all warmup
[HUGECTR][03:09:41][INFO][RANK0]: Using All-reduce algorithm: NCCL
[HUGECTR][03:09:41][INFO][RANK0]: Device 0: Tesla V100-SXM2-32GB
[HUGECTR][03:09:41][DEBUG][RANK0]: Creating Kafka lifetime service.
[HUGECTR][03:09:41][INFO][RANK0]: num of DataReader workers: 1
[HUGECTR]